In [1]:
import os
import pandas as pd

from pyspark.sql import SparkSession
from pyspark.sql.functions import col, isnull, count, when, substring, coalesce
from functools import reduce

In [2]:
# Spark 세션 생성
spark = SparkSession.builder \
    .appName("CSV to DataFrame") \
    .config("spark.driver.memory", "5g") \
    .config("spark.executor.memory", "5g") \
    .config("spark.driver.maxResultSize", "3g") \
    .getOrCreate()

In [3]:
def load_csv_to_df_spark(directory, file_names):
    """해당 디렉토리로부터 지정된 형식의 CSV 파일들을 DataFrame으로 로드"""
    
    all_dfs = []
    for idx, file_name in enumerate(file_names):
        file_path = os.path.join(directory, file_name)
        current_df = spark.read.option("multiline", "true").csv(file_path, header=True, inferSchema=True)
        all_dfs.append(current_df)

    # reduce는 순차적인 데이터에 누적 연산, union은 병합 함수, 여기선 모든 df에 대해 순차적으로 병합하는 연산
    final_df = reduce(lambda x, y: x.union(y), all_dfs)
    return final_df

In [4]:
attend_directory = "D:/DATA_PREPROCESS/FIRESTORE_DATAS/USERS"
attend_file_names = [f for f in os.listdir(attend_directory) if f.startswith('output-') and f.endswith('.csv')]
df_attend = load_csv_to_df_spark(attend_directory, attend_file_names)

In [5]:
pd.set_option('display.max_columns', None)  # 모든 열을 표시
pd.set_option('display.width', None)        # 화면 너비에 맞게 출력
print(df_attend.limit(5).toPandas())
# df_attend.count()
df_attend.columns

                        user_id        ArrayVoice attend resultPass  \
0  2A0dHXWyfYfyEbaHVsOiYA8Hlnr2        ['화이트 모던']      1       pass   
1  2A0dHXWyfYfyEbaHVsOiYA8Hlnr2  ['멋스러운 컬러 인테리어']      1       pass   
2  2A0dHXWyfYfyEbaHVsOiYA8Hlnr2       ['깔끔한 디자인']      1       pass   
3  2A0dHXWyfYfyEbaHVsOiYA8Hlnr2       ['모던한 디자인']      1       pass   
4  2A0dHXWyfYfyEbaHVsOiYA8Hlnr2      ['모던심플 스타일']      1       pass   

                                            imageUrl    addText  \
0  https://firebasestorage.googleapis.com/v0/b/sa...      화이트모던   
1  https://firebasestorage.googleapis.com/v0/b/sa...  멋스런컬러인테리어   
2  https://firebasestorage.googleapis.com/v0/b/sa...     깔끔한디자인   
3  https://firebasestorage.googleapis.com/v0/b/sa...     모던한디자인   
4  https://firebasestorage.googleapis.com/v0/b/sa...    모던심플스타일   

                                                name ArrayPercent  \
0  미자크 데일리 1900 주방 수납장 - 냉장고 키큰 부엌 틈새 팬트리 빌트인 장 자...      ['100']   
1  미자크 리버스 틈새 주방 수납장 냉장고자리 부엌 팬트

['user_id',
 'ArrayVoice',
 'attend',
 'resultPass',
 'imageUrl',
 'addText',
 'name',
 'ArrayPercent',
 'ArrayDate',
 'title',
 'type',
 'review_wish',
 'ArrayAddResult']

In [6]:
firestore_directory = "D:/DATA_PREPROCESS/FIRESTORE_DATAS"
df_memo = load_csv_to_df_spark(firestore_directory, ['fs_memo_20230803.csv'])
df_point = load_csv_to_df_spark(firestore_directory, ['fs_point_20230801.csv'])

print(f"df_memo 행 개수: {df_memo.count()} df_point 행 개수: {df_point.count()}")


# # 합치기 전에 타입 확인하기 -> point와 memo의 동일 column의 타입들이 달랐다
# print("df_memo columns:")
# for col, dtype in df_memo.dtypes:
#     print(f"{col}: {dtype}")
# print("\ndf_point_dropped columns:")
# for col, dtype in df_point.dtypes:
#     print(f"{col}: {dtype}")

# df_point의 데이터 타입 변경
df_point = df_point.withColumn("linkClick", df_point["linkClick"].cast("int")) \
                   .withColumn("linkClickDesc", df_point["linkClickDesc"].cast("int")) \
                   .withColumn("linkWing", df_point["linkWing"].cast("boolean")) \
                   .withColumn("pay", df_point["pay"].cast("int")) \
                   .withColumn("percent", df_point["percent"].cast("int")) \
                   .withColumn("player", df_point["player"].cast("int")) \
                   .withColumn("point", df_point["point"].cast("int")) \
                   .withColumn("type", df_point["type"].cast("boolean"))

# 'quiz' 컬럼 제거
df_point_dropped = df_point.drop('quiz')

# 데이터프레임 합치기
df_ad = df_memo.union(df_point_dropped)

# 결과 확인
print(f"df_ad의 행 개수: {df_ad.count()} 열 개수: {len(df_ad.columns)}")

df_memo 행 개수: 5299 df_point 행 개수: 1180
df_ad의 행 개수: 6479 열 개수: 30


In [7]:
# df_ssd = load_csv_to_df_spark(firestore_directory, ['230517_d_95_extracted_ssd_d95_data_1.csv'])
df_hdd = load_csv_to_df_spark(firestore_directory, ['230517_d_80_gcs_extracted_hdd_d80_data_1.csv', '230517_d_80_gcs_extracted_hdd_d80_data_2.csv'])
# df_externals = df_ssd.union(df_hdd)
df_externals = df_hdd
df_externals.show(5)
df_externals.count()

+--------+------------------------------+--------------+--------------------+------+----------+----------+-------+
|accuracy|                       ad_name|   record_time|             user_id|gender|birth_year|local_code|is_test|
+--------+------------------------------+--------------+--------------------+------+----------+----------+-------+
|      80|'섬·바다 여권’ 들고 여행을 ...|20221101162726|icVEsPDs4ia0m9Jjx...|     F|        89|        Gy|      0|
|      80|'섬·바다 여권’ 들고 여행을 ...|20221101162949|G5ultYhAslNplMxW0...|     M|         0|        EE|      1|
|      80|'섬·바다 여권’ 들고 여행을 ...|20221101180026|4uNFklxemxO3dnKL5...|     F|        91|        Ch|      0|
|      80|'섬·바다 여권’ 들고 여행을 ...|20221101180216|puDQ5R5fRba59WJFd...|     F|        91|        Se|      0|
|      80|'섬·바다 여권’ 들고 여행을 ...|20221101180549|BG8E6coISdOvISp84...|     F|        63|        Gy|      0|
+--------+------------------------------+--------------+--------------------+------+----------+----------+-------+
only showing to

1128340

In [8]:
df_users = load_csv_to_df_spark(firestore_directory, ['users_data.csv'])
df_users.show(5)
df_users.count()

+------------+----+--------+--------------+----+--------+------------+--------+----------+-------------------------+------------+--------------+------+-------+--------------+------------+-----+---------------+---------+---------------+-------------+--------+----+----------+--------------------+---+--------------------+----------------+------+-------------------------------+-------------+----------+--------+--------------------+------------------+--------------------+-----------------+--------+--------------+----------------+----------+----+----+-----------+-----------------+-----------+----------+----------+-----------+-----+----+
|REVIEW_TOPIC|auth|birthday|      wingTime|wing|realName|PRIZES_TOPIC|    name|     phone|isChallengeRequestAllowed|monthlyPoint| detailAddress|engine|perfect|isSoundAllowed|NOTICE_TOPIC|point|monthlyPerfects|ANY_TOPIC|MARKETING_TOPIC|searchKeyWord|language| job|      date|               image|win|                mail|         phoneId|gender|                 

34539

In [9]:
# df_users 불완전 데이터 체크

# user_id를 제외한 모든 컬럼 리스트 생성
columns_except_user_id = [c for c in df_users.columns if c != "user_id"]

# 모든 해당 컬럼들이 null인 조건 생성
condition = [isnull(column_name) for column_name in columns_except_user_id]
combined_condition = condition[0]
for c in condition[1:]:
    combined_condition &= c

# 조건에 맞는 행을 필터링하고 개수 확인
missing_rows_count = df_users.filter(combined_condition).count()
print(f"Number of rows where all columns (except user_id) are missing: {missing_rows_count}")

Number of rows where all columns (except user_id) are missing: 0


# df_attend & df_externals 조인 쿼리 수행
### df_attend가 기준 스키마

In [10]:
# df_attend와 df_externals의 조인

# df_attend에서 가져올 컬럼 선택
selected_columns = [df_attend["user_id"], df_attend["name"]]
for column in df_attend.columns:
    if column not in ["ArrayAddResult", "type", "review_wish"]:
        selected_columns.append(df_attend[column])
        
# df_externals에서 가져올 컬럼 추가
for column in ["birth_year", "local_code", "is_test"]:
    selected_columns.append(df_externals[column])

# 조인 조건 설정
join_condition_attend_externals = (df_attend["user_id"] == df_externals["user_id"]) & (df_attend["name"] == df_externals["ad_name"])

# 조인 수행
result_df_attend_externals = df_attend.join(df_externals, join_condition_attend_externals, "left_outer")

# 중복된 컬럼 제거
result_df_attend_externals = result_df_attend_externals.drop(df_externals["user_id"]).drop("accuracy").drop(df_externals["ad_name"])


first_row = result_df_attend_externals.head().asDict()
for key, value in first_row.items():
    print(f"{key}: {value}")
result_df_attend_externals.columns

user_id: 16dmLeRXkTTRLyN2mrffyYxEs5o2
ArrayVoice: ['수영만 요트투어']
attend: 1
resultPass: pass
imageUrl: https://firebasestorage.googleapis.com/v0/b/samboss-reward.appspot.com/o/ads_script%2F1%2Ft8.png?alt=media&token=b3bd5003-f4af-43e8-b845-6521af9279f2
addText: 수영만요트투어
name: [부산요트투어] 오직 우리만 타는 럭셔리 프라이빗 해운대 광안리 더베이101 파워요트 10967
ArrayPercent: ['100']
ArrayDate: ['23.07.10 오전 07시 52분']
title: 암기플러스
type: True
review_wish: 0
ArrayAddResult: ['수영만요트투어']
record_time: None
gender: None
birth_year: None
local_code: None
is_test: None


['user_id',
 'ArrayVoice',
 'attend',
 'resultPass',
 'imageUrl',
 'addText',
 'name',
 'ArrayPercent',
 'ArrayDate',
 'title',
 'type',
 'review_wish',
 'ArrayAddResult',
 'record_time',
 'gender',
 'birth_year',
 'local_code',
 'is_test']

In [11]:
# def find_duplicate_columns(df):
#     columns = df.columns
#     duplicates = [col for col in columns if columns.count(col) > 1]
#     return list(set(duplicates))

# duplicate_columns = find_duplicate_columns(result_df_attend_externals)
# print(duplicate_columns)

In [12]:
# gender 값이 있는 행의 개수
count_with_values = result_df_attend_externals.filter(result_df_attend_externals['gender'].isNotNull()).count()

# gender 값이 없는 행의 개수
count_without_values = result_df_attend_externals.filter(result_df_attend_externals['gender'].isNull()).count()

print(f"gender에 값이 있는 행의 개수: {count_with_values}")
print(f"gender에 값이 없는 행의 개수: {count_without_values}")

gender에 값이 있는 행의 개수: 1010475
gender에 값이 없는 행의 개수: 5703556


In [13]:
non_null_externals_rows = df_externals.filter(
    (df_externals["birth_year"].isNotNull()) |
    (df_externals["local_code"].isNotNull()) |
    (df_externals["is_test"].isNotNull())
).count()
print(f"df_externals에서 'birth_year', 'local_code', 'is_test' 중 하나라도 null이 아닌 행의 수: {non_null_externals_rows}")

# 2단계: result_df_attend_externals에서 'birth_year', 'local_code', 'is_test' 중 하나라도 null인 행의 수 확인
non_null_result_rows = result_df_attend_externals.filter(
    (result_df_attend_externals["birth_year"].isNotNull()) |
    (result_df_attend_externals["local_code"].isNotNull()) |
    (result_df_attend_externals["is_test"].isNotNull())
).count()

print(f"result_df_attend_externals에서 'birth_year', 'local_code', 'is_test' 중 하나라도 null이 아닌 행의 수: {non_null_result_rows}")

df_externals에서 'birth_year', 'local_code', 'is_test' 중 하나라도 null이 아닌 행의 수: 1128340
result_df_attend_externals에서 'birth_year', 'local_code', 'is_test' 중 하나라도 null이 아닌 행의 수: 1010475


In [14]:
# df_attend와 df_externals에서 조인 조건에 맞는 키 값이 얼마나 많이 일치하는지 확인
matching_keys_count = df_attend.join(df_externals, join_condition_attend_externals, "inner").count()
print(f"Matching keys between df_attend and df_externals: {matching_keys_count}")

Matching keys between df_attend and df_externals: 1010475


In [15]:
# df_externals 내에서 user_id와 ad_name의 조합이 중복으로 나타나는 경우의 수
duplicate_keys_count = df_externals.groupBy("user_id", "ad_name").agg(count("*").alias("num_rows")).filter("num_rows > 1").count()
print(f"df_externals 내에서 user_id와 ad_name의 조합이 중복으로 나타나는 수: {duplicate_keys_count}")

# df_externals 내에서 user_id와 ad_name, accuracy의 조합이 중복으로 나타나는 경우의 수
duplicate_keys_count = df_externals.groupBy("user_id", "ad_name", "accuracy").agg(count("*").alias("num_rows")).filter("num_rows > 1").count()
print(f"df_externals 내에서 user_id와 ad_name, accuracy의 조합이 중복으로 나타나는 수: {duplicate_keys_count}")

# df_externals 내에서 user_id와 ad_name, record_time의 조합이 중복으로 나타나는 경우의 수
duplicate_keys_count = df_externals.groupBy("user_id", "ad_name", "record_time").agg(count("*").alias("num_rows")).filter("num_rows > 1").count()
print(f"df_externals 내에서 user_id, ad_name, 그리고 record_time의 조합이 중복으로 나타나는 수: {duplicate_keys_count}")

# df_externals 내에서 user_id와 ad_name, gender의 조합이 중복으로 나타나는 경우의 수
duplicate_keys_count = df_externals.groupBy("user_id", "ad_name", "gender").agg(count("*").alias("num_rows")).filter("num_rows > 1").count()
print(f"df_externals 내에서 user_id, ad_name, 그리고 gender의 조합이 중복으로 나타나는 수: {duplicate_keys_count}")

# df_externals 내에서 user_id와 ad_name, birth_year의 조합이 중복으로 나타나는 경우의 수
duplicate_keys_count = df_externals.groupBy("user_id", "ad_name", "birth_year").agg(count("*").alias("num_rows")).filter("num_rows > 1").count()
print(f"df_externals 내에서 user_id, ad_name, 그리고 birth_year의 조합이 중복으로 나타나는 수: {duplicate_keys_count}")

# df_externals 내에서 record_time을 제외한 파일명이 중복으로 나타나는 경우의 수
duplicate_keys_count = df_externals.groupBy("user_id", "ad_name", "gender", "birth_year", "local_code").agg(count("*").alias("num_rows")).filter("num_rows > 1").count()
print(f"df_externals 내에서 record_time을 제외한 파일명이 중복으로 나타나는 경우의 수: {duplicate_keys_count}")

df_externals 내에서 user_id와 ad_name의 조합이 중복으로 나타나는 수: 5189
df_externals 내에서 user_id와 ad_name, accuracy의 조합이 중복으로 나타나는 수: 5189
df_externals 내에서 user_id, ad_name, 그리고 record_time의 조합이 중복으로 나타나는 수: 0
df_externals 내에서 user_id, ad_name, 그리고 gender의 조합이 중복으로 나타나는 수: 5186
df_externals 내에서 user_id, ad_name, 그리고 birth_year의 조합이 중복으로 나타나는 수: 5181
df_externals 내에서 record_time을 제외한 파일명이 중복으로 나타나는 경우의 수: 5180


## 일단 외부 저장소(SSD, HDD)에 데이터가 중복 저장되었다고 간주, 다음 작업으로 넘어간다

df_externals 내에서 user_id와 ad_name의 조합이 중복으로 나타나는 수: 373451
df_externals 내에서 user_id, ad_name, 그리고 gender의 조합이 중복으로 나타나는 수: 373450
df_externals 내에서 user_id, ad_name, 그리고 birth_year의 조합이 중복으로 나타나는 수: 373446

이 결과를 통해 user_id, ad_name, gender, birth_year 즉 SSD에 저장된 record_time이 없는 거의 대부분의 데이터가 HDD 데이터와 중복이 발생한다는 걸 확인할 수 있다.
추가 작업을 통해 이 중복이 확실한 것인지 체크하고 관련해서 활동들이 필요하다

user 수 := ssd u95 파일 수 := df_externals의 중복수 
이 세가지의 수가 서로 근사치임을 참고

! u80은 80퍼 이상을 모두 포함하니 u95는 당연히 모두 포함되잖아?! -> 그럼 df_externals에 ssd의 u95는 포함안시키는 게 맞네

# result_df_attend_externals & df_users 조인 쿼리 수행
### result_df_attend_externals가 기준 스키마

In [16]:
# df_users 변환
df_users_transformed = df_users.withColumn("gender", when(col("gender") == "남자", "M").otherwise("W")) \
                               .withColumn("birthday", substring(col("birthday"), 3, 2)) \
                               .withColumnRenamed("user_id", "user_id_in_users") \
                               .withColumn("engine", when(col("engine") == "수도권", "Se")
                                           .when(col("engine") == "경상권", "Gy")
                                           .when(col("engine") == "강원권", "Ga")
                                           .when(col("engine") == "충청권", "Ch")
                                           .when(col("engine") == "전라권", "Jd")
                                           .when(col("engine") == "제주권", "Je")
                                           .otherwise("EE"))  # Jd가 전라도인지 제주도인지 불확실

# df_users_transformed에서 필요한 컬럼만 선택
df_users_transformed = df_users_transformed.select("user_id_in_users", "gender", "birthday", "engine", "job", "language", "perfect")

# 조인 조건 설정
join_condition = (result_df_attend_externals["user_id"] == df_users_transformed["user_id_in_users"])

# 조인 수행
result_df_users = result_df_attend_externals.join(df_users_transformed, join_condition, "left_outer")

# 필요한 컬럼만 선택하여 중복을 제거
result_df_users = result_df_users.select(result_df_attend_externals["*"],
                                         df_users_transformed["gender"].alias("new_gender"),
                                         df_users_transformed["birthday"].alias("new_birthday"),
                                         df_users_transformed["engine"].alias("new_engine"),
                                         df_users_transformed["job"],
                                         df_users_transformed["language"],
                                         df_users_transformed["perfect"])

# coalesce 함수를 사용하여 컬럼 값을 갱신
result_df_users = result_df_users.withColumn("gender", coalesce(col("new_gender"), col("gender"))) \
                                 .withColumn("birth_year", coalesce(col("new_birthday"), col("birth_year"))) \
                                 .withColumn("local_code", coalesce(col("new_engine"), col("local_code")))

# 중복된 컬럼 및 임시 컬럼 제거
result_df_users = result_df_users.drop("new_gender", "new_birthday", "new_engine", "user_id_in_users")


# 중복된 컬럼 제거
result_df_users = result_df_users.drop(df_users_transformed["user_id_in_users"])

result_df_users.show(5)

+--------------------+-------------------+------+----------+--------------------+--------------+---------------------------------+------------+-----------------------+----------+----+-----------+------------------+-----------+------+----------+----------+-------+------+--------+-------+
|             user_id|         ArrayVoice|attend|resultPass|            imageUrl|       addText|                             name|ArrayPercent|              ArrayDate|     title|type|review_wish|    ArrayAddResult|record_time|gender|birth_year|local_code|is_test|   job|language|perfect|
+--------------------+-------------------+------+----------+--------------------+--------------+---------------------------------+------------+-----------------------+----------+----+-----------+------------------+-----------+------+----------+----------+-------+------+--------+-------+
|16dmLeRXkTTRLyN2m...|['수영만 요트투어']|     1|      pass|https://firebases...|수영만요트투어| [부산요트투어] 오직 우리만 타...|     ['100']|['23.07.10 오전 07시..

In [17]:
# 각 값에 따른 개수 출력
gender_counts = result_df_users.groupBy("gender").count().show()

+------+-------+
|gender|  count|
+------+-------+
|     F|  23445|
|  null|  83857|
|     M|1405604|
|     W|5201125|
+------+-------+



In [18]:
# name 값이 있는 행의 개수
name_present_count = result_df_users.filter(result_df_users.name.isNotNull()).count()

# name 값이 없는 행의 개수
name_absent_count = result_df_users.filter(result_df_users.name.isNull()).count()

# # ArrayDate 값이 없으면서 record_time에 값이 있는 행의 개수
# array_date_absent_record_time_present_count = result_df_users.filter(
#     (result_df_users.ArrayDate.isNull()) & 
#     (result_df_users.record_time.isNotNull())
# ).count()

print(f"ArrayDate에 값이 있는 행의 개수: {name_present_count}")
print(f"ArrayDate에 값이 없는 행의 개수: {name_absent_count}")

ArrayDate에 값이 있는 행의 개수: 6714030
ArrayDate에 값이 없는 행의 개수: 1


In [19]:
print(f"전체 행의 개수: {result_df_users.count()}")

전체 행의 개수: 6714031


# result_df_users & df_ad 조인 쿼리 수행

### result_df_attend_externals가 기준 스키마¶

In [20]:
# df_point에서 컬럼 이름 변경
df_ad_renamed = df_ad.withColumnRenamed("ageArray", "excepted_age_array") \
                           .withColumnRenamed("date", "ad_duration") \
                           .withColumnRenamed("dateEnd", "ad_duration_end") \
                           .withColumnRenamed("descArray", "desc_images_array") \
                           .withColumnRenamed("imageArray", "images_array") \
                           .withColumnRenamed("imageUrl", "thumbnail_image") \
                           .withColumnRenamed("link", "ad_link") \
                           .withColumnRenamed("player", "participant_count") \
                           .withColumnRenamed("name", "ad_name")

# df_ad에서 필요한 컬럼만 선택
selected_df_ad = df_ad_renamed.select("excepted_age_array", "collection", "ad_duration", "ad_duration_end",
                                           "desc_images_array", "images_array", "thumbnail_image", "level",
                                           "ad_link", "participant_count", "videoUrl", "ad_name")

# 조인 수행
join_condition_ad = (result_df_users["name"] == selected_df_ad["ad_name"])
result_df = result_df_users.join(selected_df_ad, join_condition_ad, "left_outer")

# 중복 컬럼 제거
result_df = result_df.drop(selected_df_ad["ad_name"])

# 최종 결과 출력
result_df.columns

['user_id',
 'ArrayVoice',
 'attend',
 'resultPass',
 'imageUrl',
 'addText',
 'name',
 'ArrayPercent',
 'ArrayDate',
 'title',
 'type',
 'review_wish',
 'ArrayAddResult',
 'record_time',
 'gender',
 'birth_year',
 'local_code',
 'is_test',
 'job',
 'language',
 'perfect',
 'excepted_age_array',
 'collection',
 'ad_duration',
 'ad_duration_end',
 'desc_images_array',
 'images_array',
 'thumbnail_image',
 'level',
 'ad_link',
 'participant_count',
 'videoUrl']

In [21]:
# print(f"전체 행의 개수: {result_df_point.count()}")

# ad_duration_end 값이 있는 행의 개수
ad_duration_end_present_count = result_df.filter(result_df.ad_duration_end.isNotNull()).count()

# ad_duration_end 값이 없는 행의 개수
ad_duration_end_absent_count = result_df.filter(result_df.ad_duration_end.isNull()).count()

# # ArrayDate 값이 없으면서 record_time에 값이 있는 행의 개수
# array_date_absent_record_time_present_count = result_df_users.filter(
#     (result_df_users.ArrayDate.isNull()) & 
#     (result_df_users.record_time.isNotNull())
# ).count()

print(f"ad_duration_end에 값이 있는 행의 개수: {ad_duration_end_present_count}")
print(f"ad_duration_end에 값이 없는 행의 개수: {ad_duration_end_absent_count}")

ad_duration_end에 값이 있는 행의 개수: 3787257
ad_duration_end에 값이 없는 행의 개수: 2926774


In [22]:
# os.environ['HADOOP_HOME'] = "C:\\Hadoop\\hadoop-3.3.6"
print(os.environ["HADOOP_HOME"])

C:\Hadoop\hadoop-3.3.6


In [ ]:
result_df.show()

In [23]:
result_df.coalesce(10).write.csv("D:/DATA_PREPROCESS/FIRESTORE_DATAS/voice_metadata_230829", mode='overwrite', header=True)